# Finding Anomalous gas absorbtion in MORFs
Under certain circumstances the amount of gas stored in a MORF can increase as temperature increases, this is very cool as it never occurs in a MOF without a ring.

Starting with the grand canonical ensemble, the  probability of a state is as follows:
$$P_i =\displaystyle\sum^{all states} \frac{e^{\beta(\mu N_i-E_i)}}{Z}$$

Assuming that there are four possibe states, corresponding to two ring positions and the prescence of gas the probabilty of a state can be written as:
$$P(n,r)=\frac{e^{\beta(\mu n − r\delta−n(r\epsilon_{\heartsuit} + (1−r)\epsilon_{\triangle}}}{1 +  e^{−\beta \delta} + e^{\beta(\mu−\epsilon_{\triangle})} + e^{\beta(\mu − \epsilon_{\heartsuit} − \delta)}}$$

This makes the probability of a state in our MORF:
$$P(n,r)=\frac{e^{\beta(\mu n − r\delta−n(r\epsilon_{\heartsuit} + (1−r)\epsilon_{\triangle}}}{1 +  e^{−\beta \delta} + e^{\beta(\mu−\epsilon_{\triangle})} + e^{\beta(\mu − \epsilon_{\heartsuit} − \delta)}}$$

Uisng the concept of thermal averages, the expectation value of n is:
$$<n> = \frac{e^{\beta(\mu−\epsilon_{\triangle})} + e^{\beta(\mu − \epsilon_{\heartsuit} − \delta)}}{1 +  e^{−\beta \delta} + e^{\beta(\mu−\epsilon_{\triangle})} + e^{\beta(\mu − \epsilon_{\heartsuit} − \delta)}}$$

Which can in turn be written as 
$$\langle n \rangle = \frac{\lambda k}{1+\lambda k}$$

where $\lambda = e^{\beta \mu}$ and  $k = \frac{e^{−\beta \epsilon_\triangle} + e^{− \beta(\delta + \epsilon_\heartsuit))}}{1+e^{−\beta \delta}}$. 

For the correct choice of $\delta$ , $\epsilon_\heartsuit$, and $\epsilon_\triangle$ the derivative of $\langle n \rangle$ with regards to temperature, as we will now show. Note that we will use the $\beta$ derivative as it is mathematically less clunky and conveniently unitless.

Let's find $\frac{\partial \langle n \rangle}{\partial \beta} < 0$

In [1]:
using ForwardDiff
using Printf
using PyPlot
PyPlot.matplotlib.style.use("Solarize_Light2") # dope style

struct Material
    δ::Float64
    ϵ♡::Float64
    ϵΔ::Float64
end

function Base.show(io::IO, material::Material)
    @printf("MORF material with:\n\tδ = %.3f\n\tϵ♡ = %.3f\n\tϵΔ = %.3f\n", material.δ, material.ϵ♡, material.ϵΔ)
end

function K(β::Real, material::Material) # must assert β::Real as opposed to β::Float64 for ForwardDiff.jl to work
    p = 1 / (1 + exp(-β * material.δ)) # probability ring is on ♡ when gas is absent
    return p * exp(-β * material.ϵΔ) + (1 - p) * exp(-β * material.ϵ♡)
end

function λ(μ::Real, β::Real) # must assert β::Real as opposed to β::Float64 for ForwardDiff.jl to work
    return exp(β*μ)
end
    
function N(μ::Real,β::Real, material::Material)# must assert β::Real as opposed to β::Float64 for ForwardDiff.jl to work
    numerator = λ(μ,β)*K(β,material)
    denominator = 1 + λ(μ,β)*K(β,material)
    return numerator/denominator
end

N (generic function with 1 method)

Now in order to identify where in material space anomalous behavior occurs we'll utilize automatic differentation to find $\frac{\partial \langle n \rangle}{\partial \beta}$.

In [5]:
function ∂N_∂β(μ,β::Float64, material::Material)#using ⟨n⟩ in the function name triggers a syntax errror
    # view K as a function of β only.
    N_of_β(x) = N(μ, x[1], material) # x plays role of β
    ∂N_∂β_of_β = x -> ForwardDiff.gradient(N_of_β, x) # x plays role of β
    return ∂N_∂β_of_β([β])[1]
end

∂N_∂β (generic function with 2 methods)

In [7]:
nb_pts = 15
δ_range = range(0.0, stop=4.0, length=nb_pts)
ϵ♡_range = range(-4.0, stop=0.0, length=nb_pts)
ϵΔ_range = range(-4.0, stop=0.0, length=nb_pts)
β_range = range(0.01, stop=10.0, length=nb_pts)
μ_range = range(0.01, stop=5.0, length=nb_pts)

for δ in δ_range
    for ϵ♡ in ϵ♡_range
        for ϵΔ in ϵΔ_range
            material = Material(δ, ϵ♡, ϵΔ)
            for β in β_range
                for μ in μ_range
                    if ∂N_∂β(μ,β, material)<0.0
                        @printf("∂⟨n⟩/∂β < 0 when δ = %f ϵ♡ = %.2f ϵΔ = %.2f β = %.2f μ = %.2f \n", δ, ϵ♡ , ϵΔ, β, μ)
                    end
                end
            end
        end
    end
end

∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 4.29 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 5.72 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 6.43 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 7.15 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 7.86 μ = 0.72 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 8.57 μ = 2.15 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 8.57 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 8.57 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 8.57 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 9.29 μ = 1.79 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 9.29 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 10.00 μ = 0.72 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -4.00 β = 10.00 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -3.71 β = 5.00 μ = 5.00 
∂⟨n⟩

∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.57 β = 9.29 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.57 β = 10.00 μ = 1.79 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 5.72 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 6.43 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 6.43 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 6.43 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 7.15 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 7.86 μ = 1.44 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 7.86 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 8.57 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 8.57 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 9.29 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 10.00 μ = 0.01 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -4.00 ϵΔ = -0.29 β = 10.00 μ = 0.72 
∂⟨n

∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.43 β = 7.15 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.43 β = 7.86 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.43 β = 8.57 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.43 β = 8.57 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.43 β = 9.29 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 5.72 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 6.43 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 7.15 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 8.57 μ = 1.79 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 8.57 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 9.29 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 9.29 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -1.14 β = 10.00 μ = 0.72 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.71 ϵΔ = -0.86 β = 5.00 μ = 4.64 
∂⟨n⟩/

∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 7.86 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 7.86 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 8.57 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 8.57 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 8.57 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 9.29 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 10.00 μ = 0.72 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 10.00 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 10.00 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.29 β = 10.00 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.00 β = 5.00 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.00 β = 6.43 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.00 β = 7.15 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.43 ϵΔ = -2.00 β = 7.15 μ = 3.57 
∂⟨

∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.86 β = 6.43 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.86 β = 6.43 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.86 β = 10.00 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.86 β = 10.00 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.86 β = 10.00 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.57 β = 6.43 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.57 β = 7.86 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.57 β = 8.57 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.57 β = 10.00 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.29 β = 5.00 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.29 β = 5.72 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.29 β = 7.86 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.29 β = 8.57 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.000000 ϵ♡ = -3.14 ϵΔ = -2.29 β = 9.29 μ = 2.86 
∂⟨

...

∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -4.00 β = 7.86 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -4.00 β = 8.57 μ = 2.15 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -4.00 β = 8.57 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -4.00 β = 9.29 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -4.00 β = 10.00 μ = 2.15 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -4.00 β = 10.00 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -4.00 β = 10.00 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -3.71 β = 5.72 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -3.71 β = 6.43 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -3.71 β = 7.15 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -3.71 β = 8.57 μ = 0.72 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -3.71 β = 9.29 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -3.71 β = 9.29 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -1.14 ϵΔ = -3.71 β = 10.00 μ = 0.72 
∂⟨

∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -3.14 β = 9.29 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -3.14 β = 10.00 μ = 0.72 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -3.14 β = 10.00 μ = 1.79 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -3.14 β = 10.00 μ = 2.86 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 5.72 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 5.72 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 6.43 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 7.15 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 8.57 μ = 2.15 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 8.57 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 8.57 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 10.00 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.86 β = 10.00 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.86 ϵΔ = -2.57 β = 5.72 μ = 5.00 
∂

∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 7.86 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 7.86 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 8.57 μ = 3.22 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 9.29 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 9.29 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 10.00 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 10.00 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -2.00 β = 10.00 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -1.71 β = 7.15 μ = 5.00 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -1.71 β = 9.29 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -1.14 β = 7.15 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -1.14 β = 7.86 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -1.14 β = 8.57 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = -1.14 β = 10.00 μ = 3.57 
∂⟨

Excessive output truncated after 524308 bytes.

57 ϵΔ = 0.00 β = 8.57 μ = 0.01 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = 0.00 β = 9.29 μ = 0.01 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = 0.00 β = 10.00 μ = 0.01 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.57 ϵΔ = 0.00 β = 10.00 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 5.00 μ = 3.93 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 5.72 μ = 3.57 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 6.43 μ = 4.64 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 7.15 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 7.15 μ = 4.29 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 7.86 μ = 1.08 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 7.86 μ = 1.79 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 7.86 μ = 2.50 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 8.57 μ = 1.08 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = -4.00 β = 8.57 μ = 1.44 
∂⟨n⟩/∂β > 0 when δ = 0.857143 ϵ♡ = -0.29 ϵΔ = 